In [1]:
import tensorflow as tf
import numpy as np
import os
import time

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from sklearn.metrics import classification_report

In [2]:
# PATH DATASET
DATASET_DIR = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Dataset_TrashNet_Final"
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
TEST_DIR  = os.path.join(DATASET_DIR, "test")

# PATH MODEL
SAVED_MODEL_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Pruning/mobilenetv3Large_trashnet_pruned_20-30-40"
TFLITE_INT16_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Final/pruned-mobilenetv3large_ptq_int16.tflite"

# PARAMETER
IMG_SIZE = (224, 224)
BATCH_SIZE = 1
# CALIBRATION_SAMPLES = 100 tidak diperlukan karna yang di-quantize hanya weight

In [3]:
model_fp32 = tf.keras.models.load_model(
    SAVED_MODEL_PATH,
    compile=False
)

print("Model FP32 berhasil dimuat.")




Model FP32 berhasil dimuat.


In [4]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

class_names = list(test_generator.class_indices.keys())

Found 2001 images belonging to 6 classes.
Found 383 images belonging to 6 classes.


In [5]:
# SETUP CONVERTER PTQ INT16
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)

# Optimasi default
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Quantize weight ke INT16
converter.target_spec.supported_types = [tf.int16]

In [6]:
# KONVERSI & SIMPAN MODEL INT16
print("Melakukan Post-Training Quantization INT16...")
tflite_int16_model = converter.convert()

with open(TFLITE_INT16_PATH, "wb") as f:
    f.write(tflite_int16_model)

model_size_mb = os.path.getsize(TFLITE_INT16_PATH) / (1024 * 1024)

print("Model INT16 berhasil disimpan.")
print(f"Ukuran model INT16: {model_size_mb:.2f} MB")

Melakukan Post-Training Quantization INT16...
Model INT16 berhasil disimpan.
Ukuran model INT16: 11.59 MB


In [7]:
# LOAD INTERPRETER TFLITE
interpreter = tf.lite.Interpreter(model_path=TFLITE_INT16_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input dtype :", input_details[0]["dtype"])
print("Output dtype:", output_details[0]["dtype"])

Input dtype : <class 'numpy.float32'>
Output dtype: <class 'numpy.float32'>


In [8]:
# EVALUASI MODEL (DATA TEST)
y_true = []
y_pred = []

start_time = time.time()

for i in range(len(test_generator)):
    image, label = test_generator[i]

    interpreter.set_tensor(
        input_details[0]["index"], image.astype(np.float32)
    )
    interpreter.invoke()

    output = interpreter.get_tensor(
        output_details[0]["index"]
    )

    y_true.append(np.argmax(label))
    y_pred.append(np.argmax(output))

end_time = time.time()

In [9]:
# LAPORAN WAKTU INFERENSI
total_time = end_time - start_time
avg_inference_time = total_time / len(test_generator)

print("Time Report PTQ INT16")
print(f"Total inference time  : {total_time:.4f} detik")
print(f"Rata-rata per gambar  : {avg_inference_time:.6f} detik")

Time Report PTQ INT16
Total inference time  : 4.1690 detik
Rata-rata per gambar  : 0.010885 detik


In [10]:
# CLASSIFICATION REPORT
print("Classification Report PTQ INT16")
print(
    classification_report(
        y_true,
        y_pred,
        target_names=class_names,
        digits=4
    )
)

Classification Report PTQ INT16
              precision    recall  f1-score   support

   cardboard     0.9123    0.8525    0.8814        61
       glass     0.8333    0.9211    0.8750        76
       metal     0.8571    0.8710    0.8640        62
       paper     0.8791    0.8889    0.8840        90
     plastic     0.8310    0.8194    0.8252        72
       trash     0.8235    0.6364    0.7179        22

    accuracy                         0.8590       383
   macro avg     0.8561    0.8315    0.8412       383
weighted avg     0.8595    0.8590    0.8580       383

